# Pre trained network
![imagen](https://www.researchgate.net/publication/336874848/figure/fig1/AS:819325225144320@1572353764073/Illustrations-of-transfer-learning-a-neural-network-is-pretrained-on-ImageNet-and.png)

Estas son las arquitecturas de redes neuronales más utilizadas en la comunidad. Para más detalle sobre el funcionamiento de cada red, consultar el [Hands on Machine Learning for Python](https://learning.oreilly.com/library/view/hands-on-machine-learning/9781492032632/ch14.html#cnn_chapter).
* VGG-16
* VGG-19
* Inception V3
* XCeption
* ResNet-50

Las redes se pueden incorporar entrenadas, o sin entrenar.

## ResNet50V2

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [2]:

'''
imagenet es un corpus de imagenes con el que se ha clasificado el ResNet50V2
http://www.image-net.org/

Cargamos toda la red ya entrenada, y la usaremos para predecir
include_top=True --> Para que incluya la fully connected layer.
include_top=False --> Desarrollamos la fully connected layer
ojo el input shape que sea el de las imagenes que introduciremos. Esta limitado a imagenes de tamaño n
classifier_activation se usa si include_top=True
'''

base_model = ResNet50V2(input_shape=(224, 224,3),
                        include_top=True,
                        weights="imagenet",
                        classifier_activation="softmax")

102869336/102869336 [==============================] - 26s 0us/step


In [3]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

Cargamos algunas imagenes desde local, para ver qué tal funciona la red ResNet50V2 ya entrenada.

In [4]:
import os


def read_data(path):
    X = []

    for file in os.listdir(path):
        image = imread(path + '/' + file)
        smallimage = cv2.resize(image, (224, 224))
        print(path + '/' + file)
        
        X.append(smallimage)

    return np.array(X)
    

x_test = read_data('img')

# Procesar las imagenes tal y como entran en el modelo
x_test = preprocess_input(x_test)
print(x_test.shape)

img/bear-1.jpg
img/cat.8016.jpg
img/cat.8037.jpg
img/dog.11856.jpg
img/dog.11857.jpg
img/horse.jpg
img/karate.jpg
img/pizza.jpg
(8, 224, 224, 3)


In [5]:
preds = base_model.predict(x_test)

# decode the results into a list of tuples (class, description, probability)
decodes = decode_predictions(preds, top=5)

for j in decodes:
    print('####################')
    for i, decode in enumerate(j):
        print('Predicted:\n', decode[1], decode[2])

35363/35363 [==============================] - 0s 0us/step
####################
Predicted:
 brown_bear 0.9994438
Predicted:
 chow 0.0005415565
Predicted:
 American_black_bear 8.780987e-06
Predicted:
 ice_bear 1.1076165e-06
Predicted:
 howler_monkey 4.4061557e-07
####################
Predicted:
 Egyptian_cat 0.64984995
Predicted:
 Siamese_cat 0.16787332
Predicted:
 tiger_cat 0.06044944
Predicted:
 lynx 0.024389775
Predicted:
 tabby 0.017880328
####################
Predicted:
 Egyptian_cat 0.8209842
Predicted:
 lynx 0.12679452
Predicted:
 Siamese_cat 0.021530585
Predicted:
 tabby 0.020569624
Predicted:
 tiger_cat 0.0074053896
####################
Predicted:
 Rottweiler 0.7329093
Predicted:
 Brabancon_griffon 0.08508471
Predicted:
 Staffordshire_bullterrier 0.03105943
Predicted:
 EntleBucher 0.023099247
Predicted:
 Doberman 0.019871553
####################
Predicted:
 collie 0.99525696
Predicted:
 Shetland_sheepdog 0.004742966
Predicted:
 yellow_lady's_slipper 9.14254e-09
Predicted:
 gold

## VGG16
En este caso vamos a importar la red VGG16, que utilizaremos como red preentrenada y completaremos con una fully connected layer. 

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

IM_SIZE=64

TRAIN_PATH = 'C:\\Users\\alber\\Downloads\\dogs-vs-cats\\mini_train'
filenames = os.listdir(TRAIN_PATH)
categories = []

for filename in filenames:
    category = filename.split('.')[0]
    categories.append(category)
    
df = pd.DataFrame({
    'filenames': filenames,
    'category': categories
})

train_df, validate_df = train_test_split(df,
                                         test_size=0.20,
                                         random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [8]:
from keras.preprocessing.image import ImageDataGenerator


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40, # Rotacion en grados
                                   width_shift_range = 0.2, # Desplazamiento a lo ancho
                                   height_shift_range = 0.2,
                                   shear_range = 0.2, # Intensidad del angulo de rotacion
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [9]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH, # Donde busscar todo esto
                                                    x_col='filenames', # Nombres de archivos
                                                    y_col='category', # Target
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 16000 validated image filenames belonging to 2 classes.
Found 4000 validated image filenames belonging to 2 classes.


In [10]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (IM_SIZE, IM_SIZE, 3), # Shape of our images
                    include_top = False, # Leave out the last fully connected layer
                    weights = 'imagenet')

# No queremos que VGG16 entrenen en el fit
for layer in base_model.layers:
    layer.trainable = False
    
##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['acc'])

58889256/58889256 [==============================] - 14s 0us/step


In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [13]:
vgghist = model.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch = 100, # Cantidad de batches en los que divide el epoch
                    epochs = 10)

Epoch 1/10
100/100 [==============================] - 59s 589ms/step - loss: 0.6716 - acc: 0.6400 - val_loss: 0.5021 - val_acc: 0.7462
Epoch 2/10
100/100 [==============================] - 72s 721ms/step - loss: 0.5840 - acc: 0.6895 - val_loss: 0.4928 - val_acc: 0.7555
Epoch 3/10
100/100 [==============================] - 76s 764ms/step - loss: 0.5516 - acc: 0.7185 - val_loss: 0.5437 - val_acc: 0.7138
Epoch 4/10
100/100 [==============================] - 81s 817ms/step - loss: 0.5597 - acc: 0.7075 - val_loss: 0.5095 - val_acc: 0.7427
Epoch 5/10
100/100 [==============================] - 78s 788ms/step - loss: 0.5302 - acc: 0.7320 - val_loss: 0.4638 - val_acc: 0.7793
Epoch 6/10
100/100 [==============================] - 69s 691ms/step - loss: 0.5544 - acc: 0.7145 - val_loss: 0.4652 - val_acc: 0.7800
Epoch 7/10
100/100 [==============================] - 70s 702ms/step - loss: 0.5285 - acc: 0.7130 - val_loss: 0.4579 - val_acc: 0.7780
Epoch 8/10
100/100 [==============================] - 7

### Inception

In [15]:
IM_SIZE=150

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 16000 validated image filenames belonging to 2 classes.
Found 4000 validated image filenames belonging to 2 classes.


In [16]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape = (IM_SIZE, IM_SIZE, 3),
                         include_top = False,
                         weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 22s 0us/step


In [17]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

inc_history = model.fit(train_generator,
                      validation_data = validation_generator,
                      steps_per_epoch = 100,
                      epochs = 10)


Epoch 1/10
100/100 [==============================] - 100s 977ms/step - loss: 9.6299 - acc: 0.8345 - val_loss: 0.5941 - val_acc: 0.9425
Epoch 2/10
100/100 [==============================] - 101s 1s/step - loss: 0.7248 - acc: 0.8880 - val_loss: 0.1086 - val_acc: 0.9575
Epoch 3/10
100/100 [==============================] - 100s 1s/step - loss: 0.3438 - acc: 0.8895 - val_loss: 0.1658 - val_acc: 0.9402
Epoch 4/10
 50/100 [==============>...............] - ETA: 23s - loss: 0.2783 - acc: 0.9030

## ResNet50V2 sin entrenar

In [133]:
IM_SIZE=32

base_model = ResNet50V2(input_shape=(IM_SIZE, IM_SIZE,3),
                        include_top=False,
                        classifier_activation="softmax")

94674944/94668760 [==============================] - 44s 0us/step


In [134]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 5260 validated image filenames belonging to 2 classes.
Found 1316 validated image filenames belonging to 2 classes.


In [135]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

history = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)


Epoch 1/10
100/100 [==============================] - 125s 1s/step - loss: 0.8080 - acc: 0.5520 - val_loss: 0.6865 - val_acc: 0.5502
Epoch 2/10
100/100 [==============================] - 126s 1s/step - loss: 0.7233 - acc: 0.5640 - val_loss: 0.6984 - val_acc: 0.6071
Epoch 3/10
100/100 [==============================] - 130s 1s/step - loss: 0.6792 - acc: 0.6060 - val_loss: 0.6775 - val_acc: 0.6406
Epoch 4/10
100/100 [==============================] - 130s 1s/step - loss: 0.6812 - acc: 0.6265 - val_loss: 0.6875 - val_acc: 0.6474
Epoch 5/10
100/100 [==============================] - 129s 1s/step - loss: 0.6738 - acc: 0.6290 - val_loss: 1.8932 - val_acc: 0.6421
Epoch 6/10
100/100 [==============================] - 130s 1s/step - loss: 0.6732 - acc: 0.6320 - val_loss: 1.5917 - val_acc: 0.6307
Epoch 7/10
100/100 [==============================] - 127s 1s/step - loss: 0.6613 - acc: 0.6405 - val_loss: 5.9469 - val_acc: 0.6368
Epoch 8/10
100/100 [==============================] - 123s 1s/step - 

In [136]:
history = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)


Epoch 1/10
100/100 [==============================] - 112s 1s/step - loss: 0.6270 - acc: 0.6780 - val_loss: 1.0451 - val_acc: 0.7143
Epoch 2/10
100/100 [==============================] - 113s 1s/step - loss: 0.6042 - acc: 0.6780 - val_loss: 0.9609 - val_acc: 0.7340
Epoch 3/10
100/100 [==============================] - 113s 1s/step - loss: 0.6506 - acc: 0.6730 - val_loss: 1.0535 - val_acc: 0.7386
Epoch 4/10
100/100 [==============================] - 113s 1s/step - loss: 0.6320 - acc: 0.6695 - val_loss: 0.5738 - val_acc: 0.7530
Epoch 5/10
100/100 [==============================] - 113s 1s/step - loss: 0.6159 - acc: 0.6930 - val_loss: 0.9917 - val_acc: 0.7515
Epoch 6/10
100/100 [==============================] - 113s 1s/step - loss: 0.6194 - acc: 0.7015 - val_loss: 0.6849 - val_acc: 0.7356
Epoch 7/10
100/100 [==============================] - 113s 1s/step - loss: 0.5911 - acc: 0.7140 - val_loss: 0.7470 - val_acc: 0.7508
Epoch 8/10
100/100 [==============================] - 113s 1s/step - 